In [1]:
gedicht = """
Wer reitet so spät durch Nacht und Wind?
Es ist der Vater mit seinem Kind;
Er hat den Knaben wohl in dem Arm,
Er fasst ihn sicher, er hält ihn warm.

Mein Sohn, was birgst du so bang dein Gesicht? –
Siehst, Vater, du den Erlkönig nicht?
Den Erlenkönig mit Kron’ und Schweif? –
Mein Sohn, es ist ein Nebelstreif. –

„Du liebes Kind, komm, geh mit mir!
Gar schöne Spiele spiel’ ich mit dir;
Manch’ bunte Blumen sind an dem Strand,
Meine Mutter hat manch gülden Gewand.“ –

Mein Vater, mein Vater, und hörest du nicht,
Was Erlenkönig mir leise verspricht? –
Sei ruhig, bleibe ruhig, mein Kind;
In dürren Blättern säuselt der Wind. –

„Willst, feiner Knabe, du mit mir gehn?
Meine Töchter sollen dich warten schön;
Meine Töchter führen den nächtlichen Reihn
Und wiegen und tanzen und singen dich ein.“ –

Mein Vater, mein Vater, und siehst du nicht dort
Erlkönigs Töchter am düstern Ort? –
Mein Sohn, mein Sohn, ich seh’ es genau:
Es scheinen die alten Weiden so grau. –

„Ich liebe dich, mich reizt deine schöne Gestalt;
Und bist du nicht willig, so brauch’ ich Gewalt.“ –
Mein Vater, mein Vater, jetzt fasst er mich an!
Erlkönig hat mir ein Leids getan! –

Dem Vater grauset’s; er reitet geschwind,
Er hält in den Armen das ächzende Kind,
Erreicht den Hof mit Mühe und Not;
In seinen Armen das Kind war tot
"""

In [2]:
from transformers import pipeline
import numpy as np
import scipy
from IPython.display import Audio
from transformers import AutoTokenizer, AutoModelForTextToWaveform, AutoProcessor, AutoModel
import warnings
warnings.filterwarnings('ignore')
import torch
import scipy

/home/changgeng/anaconda3/envs/torchi/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
params = {
    "max_length": 256,
    "min_length": 8,
    "no_repeat_ngram_size": 3,
    "early_stopping": True,
    "repetition_penalty": 3.5,
    "length_penalty": 0.3,
    "encoder_no_repeat_ngram_size": 3,
    "num_beams": 4,
}

In [3]:
processor = AutoProcessor.from_pretrained("suno/bark-small")
model = AutoModel.from_pretrained("suno/bark-small")
# model

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

CUDA is available. Using GPU.


In [5]:
list(gedicht.split('\n'))[1:5]

['Wer reitet so spät durch Nacht und Wind?',
 'Es ist der Vater mit seinem Kind;',
 'Er hat den Knaben wohl in dem Arm,',
 'Er fasst ihn sicher, er hält ihn warm.']

In [6]:
gedicht = list(gedicht.split('\n'))[1:5]

In [7]:
for i in gedicht:
    inputs = processor(
        text=["MAN: [laughs]" + i],
        return_tensors="pt"
    )
    speech_values = model.generate(**inputs, do_sample=True).to(device)
    

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generati

In [8]:
model.generate(**inputs, pad_token_id=processor.eos_token_id)

AttributeError: 'BarkProcessor' object has no attribute 'eos_token_id'

In [ ]:
print(speech_values.dtype)
speech_values.cpu().numpy()

In [ ]:
pppath = "gedicht.wav"
scipy.io.wavfile.write(pppath, rate=24000, data=speech_values.cpu().numpy().T)
display(Audio(pppath, autoplay=True))

In [ ]:
synthesiser = pipeline("text-to-speech", "suno/bark-small")
audio_list = []
for satz in gedicht:
    speech = synthesiser(satz, forward_params={"do_sample": True})

In [ ]:
'''
synthesiser = pipeline("text-to-speech", "suno/bark-small")
speech = synthesiser(gedicht, forward_params={"do_sample": True})
'''

In [ ]:
speech

In [ ]:
ppath = "bark_out.wav"
scipy.io.wavfile.write(ppath, rate=speech["sampling_rate"], data=speech["audio"].T)
display(Audio(ppath, autoplay=True))